# Point Cloud Tutorial

In this notebook we will cover:
1. Loading RGBD data and creating an [Open3D](http://www.open3d.org/) point cloud
2. Removing points with inaccurate depth
3. Cropping and downsampling a point cloud
4. Getting the highest, lowest and random points.
5. Project points from 3D to 2D

We will use data of a robot holding a shirt in the air, prerecorded with a ZED 2i.

> Download this data from [here](https://ugentbe-my.sharepoint.com/:f:/g/personal/victorlouis_degusseme_ugent_be/EkIZoyySsnZBg56hRq1BqdkBuGlvhAwPWT9HDuqaUB-psA?e=iSehj6) and save the folder in a folder called `data` relative to this notebook.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
data_dir = os.path.join("data", "competition_sample_0000")

In [ ]:
!ls $data_dir

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from ipywidgets import interact

from airo_typing import NumpyDepthMapType
from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics
from airo_dataset_tools.data_parsers.pose import Pose
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud, crop_point_cloud
from airo_camera_toolkit.point_clouds.visualization import open3d_point
from airo_camera_toolkit.reprojection import project_frame_to_image_plane
from airo_typing import PointCloudPositionsType

In [ ]:
intrinsics_path = os.path.join(data_dir, "intrinsics.json")
image_left_path = os.path.join(data_dir, "image_left.png")
image_right_path = os.path.join(data_dir, "image_right.png")
depth_map_path = os.path.join(data_dir, "depth_map.tiff")
confidence_map_path = os.path.join(data_dir, "confidence_map.tiff")
point_cloud_path = os.path.join(data_dir, "pointcloud.ply")
camera_pose_path = os.path.join(data_dir, "camera_pose.json")

## 1. Loading the data

### 1.1 Loading the camera parameters

In [ ]:
with open(intrinsics_path, "r") as f:
    intrinsics_model = CameraIntrinsics.model_validate_json(f.read())
    intrinsics = intrinsics_model.as_matrix()
    resolution = intrinsics_model.image_resolution.as_tuple()

with open(camera_pose_path, "r") as f:
    camera_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()

with np.printoptions(precision=3, suppress=True):
    print("Resolution:", resolution)
    print("Intrinsics: \n", intrinsics)
    print("Extrinsics: \n", camera_pose)

### 1.2 Loading the color images

In [ ]:
image_left = plt.imread(image_left_path) # you can also use cv2.imread but then you get BGR instead of RGB
image_right = plt.imread(image_right_path)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(image_left)
plt.title("Left image")
plt.subplot(1, 2, 2)
plt.imshow(image_right)
plt.title("Right image")
plt.show()

### 1.3 Loading the depth and confidence map

> Note: the confidence map has range [0.0, 100.0] where 0.0 is the **most confident**.

In [ ]:
depth_map = cv2.imread(depth_map_path, cv2.IMREAD_ANYDEPTH)
confidence_map = cv2.imread(confidence_map_path, cv2.IMREAD_ANYDEPTH)

print("depth_map.dtype:", depth_map.dtype)
print("confidence_map.dtype:", confidence_map.dtype)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(depth_map)
plt.title("Depth map")
plt.colorbar(fraction=0.025, pad=0.04)
plt.subplot(1, 2, 2)
plt.imshow(confidence_map)
plt.title("Confidence map")
plt.colorbar(fraction=0.025, pad=0.04)
plt.show()

### 1.4 Loading the point cloud

Open3D uses the abbreviation `pcd` for their [PointCloud](http://www.open3d.org/docs/release/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud) object, we will use this too to distinguish between them and numpy point_clouds.

In [ ]:
pcd_in_camera = o3d.t.io.read_point_cloud(point_cloud_path)

# This conversion to float32 can be removed once the data is saved as float32
pcd_in_camera.point.positions = o3d.core.Tensor(pcd_in_camera.point.positions.numpy().astype(np.float32))

pcd = pcd_in_camera.transform(camera_pose) # transform to world frame (= base frame of left robot)
pcd

In [ ]:
o3d.visualization.draw_geometries([pcd.to_legacy()])

### 1.5 Numpy point clouds

Open3D provides a lot functionality for point clouds, see their [tutorial](http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html).
However sometimes you need something custom, (e.g. getting the lowest and highest points).
This can be done easily by converting the Open3D point cloud to numpy arrays.
We've also found that some operations such as filtering with a boolean mask are faster in numpy.

In [ ]:
point_cloud = open3d_to_point_cloud(pcd)
points, colors = point_cloud.points, point_cloud.colors

print("points:", points.shape, points.dtype)
if colors is not None:
    print("colors:", colors.shape, colors.dtype)

## 2. Removing low confidence points

### 2.1 Binarizing the confidence map

In [ ]:
def binarize_confidence(image: NumpyDepthMapType, threshold=50.0):
    confident = image <= threshold
    return confident


@interact(threshold=(0.0, 100.0, 1.0))
def show_confidence_binarized(threshold=50.0):
    confidence_binarized = binarize_confidence(confidence_map, threshold)
    confidence_image = confidence_binarized.astype(np.uint8) * 255
    plt.figure(figsize=(10, 5))
    plt.imshow(confidence_image, vmin=0, vmax=255)
    plt.colorbar(fraction=0.025, pad=0.04)
    plt.show()

In [ ]:
threshold = 1.0  # a value of 1.0 means only the most confidence points will be kept
confidence_binarized = binarize_confidence(confidence_map, threshold)
confidence_mask = confidence_binarized.reshape(-1)

In [ ]:
confidence_mask.shape, confidence_mask.dtype, confidence_mask[:5], 

### 2.2 Filtering with a binary map

In [ ]:
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)

n = len(point_cloud.points)
n_filtered = len(point_cloud_filtered.points)

print(f"Number of points: {n} -> {n_filtered} ({n_filtered / n * 100:.1f}%)")

In [ ]:
o3d.visualization.draw_geometries([point_cloud_to_open3d(point_cloud_filtered).to_legacy()])

## 3. Cropping and downsampling

### 3.1 Cropping

In [ ]:
bbox = (0.35, -0.3, 0.1), (0.7, 0.1, 0.95)

point_cloud_cropped = crop_point_cloud(point_cloud_filtered, bbox)

n = point_cloud.points.shape[0]
n_cropped = point_cloud_cropped.points.shape[0]

print(f"Number of points: {n} -> {n_cropped} ({n_cropped / n * 100:.1f}%)")

In [ ]:
bbox_o3d = o3d.geometry.AxisAlignedBoundingBox(*bbox)
bbox_o3d.color = (1.0, 0.0, 1.0)

o3d.visualization.draw_geometries([point_cloud_to_open3d(point_cloud_cropped).to_legacy(), bbox_o3d])

### 3.2 Downsampling

In [ ]:
pcd_cropped = point_cloud_to_open3d(point_cloud_cropped)
pcd_downsampled = pcd_cropped.voxel_down_sample(voxel_size=0.01)

n_cropped = len(pcd_cropped.point.positions)
n_downsampled = len(pcd_downsampled.point.positions)


print(f"Number of points: {n_cropped} -> {n_downsampled} ({n_downsampled / n_cropped * 100:.2f}%)")

In [ ]:
o3d.visualization.draw_geometries([pcd_downsampled.to_legacy()])

## 4. Getting the highest, lowest and random points

### 4.1 Highest and lowest points

In [ ]:
def highest_point(points: PointCloudPositionsType) -> np.ndarray:
    return points[np.argmax(points[:, 2])]

def lowest_point(points: PointCloudPositionsType) -> np.ndarray:
    return points[np.argmin(points[:, 2])]

highest = highest_point(point_cloud_cropped.points)
lowest = lowest_point(point_cloud_cropped.points)

with np.printoptions(precision=3, suppress=True):
    print("Highest point:", highest)
    print("Lowest point:", lowest)


In [ ]:
o3d.visualization.draw_geometries(
    [pcd_cropped.to_legacy(), open3d_point(highest, (0.0, 1.0, 0.0)), open3d_point(lowest, (1.0, 0.0, 0.0))]
)

### 4.2 Random points

In [ ]:
# set random seed for reproducibility
np.random.seed(0)

random_indices = np.random.choice(len(point_cloud_cropped.points), size=10, replace=False)
random_indices

In [ ]:
random_points = point_cloud_cropped.points[random_indices]

open3d_points = [open3d_point(p, (0.0, 0.0, 1.0)) for p in random_points]

o3d.visualization.draw_geometries([pcd_cropped.to_legacy(), *open3d_points])


## 5. Projection

### 5.1 Projecting points from 3D to 2D

In [ ]:
lowest_2d = project_frame_to_image_plane(
    lowest,
    intrinsics,
    np.linalg.inv(camera_pose)
).squeeze()

lowest_2d_int = np.rint(lowest_2d).astype(int)

lowest_2d, lowest_2d_int

In [ ]:
highest_2d = project_frame_to_image_plane(
    highest,
    intrinsics,
    np.linalg.inv(camera_pose),
).squeeze()

random_2d = project_frame_to_image_plane(
    random_points,
    intrinsics,
    np.linalg.inv(camera_pose),
)

highest_2d_int = np.rint(highest_2d).astype(int)
random_2d_int = np.rint(random_2d).astype(int)

In [ ]:
lowest_2d_int = np.rint(lowest_2d).astype(int)
highest_2d_int = np.rint(highest_2d).astype(int)
random_2d_int = np.rint(random_2d).astype(int)

image_annotated = image_left.copy()
cv2.circle(image_annotated, tuple(lowest_2d_int), 10, (0, 1.0, 0), thickness=2)
cv2.circle(image_annotated, tuple(highest_2d_int), 10, (1.0, 0, 0), thickness=2)
for p in random_2d_int:
    cv2.circle(image_annotated, tuple(p), 10, (0, 0, 1.0), thickness=2)

plt.figure(figsize=(10, 5))
plt.imshow(image_annotated)
plt.show()

### 5.2 Point cloud to segmented image (advanced)

In [ ]:
rgbd_image = pcd_cropped.project_to_rgbd_image(*resolution, intrinsics, extrinsics=np.linalg.inv(camera_pose))

image_rgb_float = np.asarray(rgbd_image.color)
depth_map_float = np.asarray(rgbd_image.depth).squeeze()

print(depth_map_float.shape)

# make background white where depth is 0.0
image_rgb_float[depth_map_float == 0.0] = 1.0

plt.figure(figsize=(10, 5))
plt.imshow(image_rgb_float)
plt.show()